# Etapa 0 - Processamento de Dados

In [48]:
import pickle
import os

from json import load
from tqdm.notebook import tqdm

In [49]:
from utils.incremental_encoder import IncrementalEncoder

### Dados Treinamento

In [57]:
tracks_encoder = IncrementalEncoder()

dataset     = []
for i in tqdm(range(0, 1000000, 1000)):
    _min = i
    _max = i + 999

    with open(f"..\dados\spotify_million_playlist_dataset\data\mpd.slice.{_min}-{_max}.json", 'r') as _file:
        playlists = load(_file)['playlists']

    for playlist in playlists:
        playlist_encoded_content = []
        for track in playlist['tracks']:
            track_id = tracks_encoder.fit_one(track['track_uri'])

            playlist_encoded_content.append(track_id)
        
        dataset.append((playlist['pid'], playlist['name'], playlist_encoded_content))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [58]:
if not os.path.isdir('../dados-processados/'):
    os.mkdir('../dados-processados/')

In [59]:
with open("../dados-processados/dataset.pickle", 'wb') as _file:
    pickle.dump(dataset, _file)

In [60]:
tracks_encoder.save("../dados-processados/encoding_tracks")

### Dados Desafio

In [61]:
tracks_encoder = IncrementalEncoder()
tracks_encoder.load("../dados-processados/encoding_tracks.json")

In [62]:
dataset_challenge = []

with open(f"..\dados\spotify_million_playlist_dataset_challenge\challenge_set.json", 'r') as _file:
    playlists = load(_file)['playlists']

for playlist in playlists:
    playlist_encoded_content = []
    for track in playlist['tracks']:
        track_id = tracks_encoder.fit_one(track['track_uri'])

        playlist_encoded_content.append(track_id)

    dataset_challenge.append( (playlist['pid'], playlist.get('name', None), playlist_encoded_content) )

len(dataset_challenge)

10000

In [63]:
with open("../dados-processados/dataset_challenge.pickle", 'wb') as _file:
    pickle.dump(dataset_challenge, _file)